必要なライブラリのインポート

In [1]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.chart import Reference, LineChart, Series
from openpyxl.utils.dataframe import dataframe_to_rows
from natsort import natsorted
import glob
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.drawing.line import LineProperties
from openpyxl.drawing.text import Paragraph, ParagraphProperties, CharacterProperties, RichTextProperties, Font
from openpyxl.chart.text import RichText
from openpyxl.utils.units import pixels_to_EMU
import os

グラフ凡例リスト,読み込む`csv`データリスト

In [2]:
wb = Workbook()
ws = wb.active
# 読み込むcsvリスト.凡例に使う分類ごとにパスを適宜変えること.
path_list1 = natsorted(glob.glob("**/open_noses/cough/t20/theta_0phi_5/1e5_re/*.csv"))
# 凡例に使うリスト
# 距離比較
path_list2 = natsorted(glob.glob("x*"))

# 温度比較
path_list3 = []
for f in natsorted(glob.glob('**/**/**/t*')):
    path_list3.append(os.path.split(f)[1])

# 会話or咳比較
path_list4 = []
for f in natsorted(glob.glob('**/**/c*')):
    path_list4.append(os.path.split(f)[1])

# 口呼吸or鼻呼吸比較
path_list5 = []
for f in natsorted(glob.glob('x500/o*')):
    path_list5.append(os.path.split(f)[1])

#首角度比較
path_list6 = []
for f in natsorted(glob.glob('x500/open_noses/cough/**/theta_*phi_*')):
    path_list6.append(os.path.split(f)[1])

#格子比較(感染者側の口周り格子)
path_list7 = []
for f in natsorted(glob.glob('**/**/**/**/**/lattice_*')):
    path_list7.append(os.path.split(f)[1])

`csv`データ抽出,グラフ用データ追記

In [3]:
for index in range(0,3):
    df = pd.read_csv(path_list1[index])
    df["volume[*1e-9ml]"] = df["volume[ml]"]*1e9
    Data = df[["num_drop","volume[*1e-9ml]"]]
    
    #ここまでデータの抽出
    ID = []
    ID.extend(str(i) for i in range(1,54))
    ID.extend(['LeftUp','LeftLow','RightUp','RightMid','RightLow'])
    #leftup : 2 ~ 12
    #leftlow : 13 ~ 24
    #rightup : 25 ~ 35
    #rightmid : 36 ~ 39
    #rightlow : 40 ~ 54
    Data_c = Data.copy()
    Data_c.loc["leftup"] = Data_c.iloc[0:11,:].sum()
    Data_c.loc["leftlow"] = Data_c.iloc[11:23,:].sum()
    Data_c.loc["rightup"] = Data_c.iloc[23:34,:].sum()
    Data_c.loc["rightmid"] = Data_c.iloc[34:38,:].sum()
    Data_c.loc["rightlow"] = Data_c.iloc[38:53,:].sum()
    Data_c.insert(loc = 0, column = 'ID', value = ID) 
    outputData = Data_c.iloc[53:58]

    for i in dataframe_to_rows(outputData,index=None,header=True):
        ws.append(i)

Chart1(飛沫数)基本設定

In [4]:
#c1
c1 = LineChart()

c1.y_axis.title = "number of droplets"
c1.x_axis.title = "Airway site"
c1_data_row_min_list = [2,8,14]
c1_data_row_max_list = [6,12,18]
for index in range(0,3):
    data = Reference(ws,min_col = 2,max_col=2, min_row=c1_data_row_min_list[index],max_row=c1_data_row_max_list[index])
    
    category = Reference(ws, min_col = 1, max_col=1, min_row=2, max_row= 6)
    series = Series(data, title=path_list2[index])
    c1.append(series)
    c1.set_categories(category)

Chart1オプション設定

In [5]:
#option
#ser1
ser1=c1.series[0]

ser1.marker.symbol = 'circle'     # マーカーの形状(円)を選択
ser1.marker.size = 8              # マーカーの大きさを設定(整数)
ser1.marker.graphicalProperties.solidFill = 'FF0000' # マーカーの塗り潰し色を設定
ser1.marker.graphicalProperties.line.solidFill = 'FF0000'
ser1.graphicalProperties.line.solidFill = 'FF0000' # 線の色を設定する
ser1.graphicalProperties.line.width = pixels_to_EMU(1.0) # マーカーの太さ

#ser2
ser2=c1.series[1]

ser2.marker.symbol = 'diamond'
ser2.marker.size = 8
ser2.marker.graphicalProperties.solidFill = '008000'
ser2.marker.graphicalProperties.line.solidFill = '008000'
ser2.graphicalProperties.line.solidFill = '008000'
ser2.graphicalProperties.line.width = pixels_to_EMU(1.0)

# #ser3
ser3=c1.series[2]

ser3.marker.symbol = 'square'
ser3.marker.size = 8
ser3.marker.graphicalProperties.solidFill = '0000FF'       
ser3.marker.graphicalProperties.line.solidFill = '0000FF'
ser3.graphicalProperties.line.solidFill = '0000FF'
ser3.graphicalProperties.line.width = pixels_to_EMU(1.0)

#axis
c1.x_axis.majorTickMark = "in"
c1.y_axis.majorTickMark = "in"
c1.y_axis.majorGridlines = None
c1.x_axis.majorGridlines = None
c1.graphical_properties = GraphicalProperties(ln=LineProperties(noFill=True))
c1.plot_area.spPr = GraphicalProperties(ln = LineProperties(solidFill = "000000"))
c1.x_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))
c1.y_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))

#font
font = Font(typeface='Times New Roman')
size = 1000 # 10 point size
char_prop = CharacterProperties(latin=font, sz=size, b=True, solidFill="000000") #
para_prop = ParagraphProperties(defRPr=char_prop)
c1.x_axis.title.tx.rich.p[0].pPr = para_prop
c1.y_axis.title.tx.rich.p[0].pPr = para_prop

char_prop = CharacterProperties(latin=font, sz=size, b=False) # b=Trueでボールドにする
para_prop = ParagraphProperties(defRPr=char_prop)
rich_text = RichText(p=[Paragraph(pPr=para_prop, endParaRPr=char_prop)])
c1.legend.txPr = rich_text
c1.x_axis.txPr = rich_text
c1.y_axis.txPr = rich_text

Chart2(飛沫体積)基本設定

In [6]:
#c2
c2 = LineChart()

c2.y_axis.title = "volume [*1e-9]"
c2.x_axis.title = "Airway site"

c2_data_row_min_list = [2,8,14]
c2_data_row_max_list = [6,12,18]
for index in range(0,3):
    data = Reference(ws,min_col = 3,max_col=3, min_row=c2_data_row_min_list[index],max_row=c2_data_row_max_list[index])

    category = Reference(ws, min_col = 1, max_col=1, min_row=2, max_row= 6)

    series = Series(data, title=path_list2[index])
    c2.append(series)
    c2.set_categories(category)

Chart2オプション設定

In [7]:
#option
#ser4
ser4=c2.series[0]

ser4.marker.symbol = 'circle'
ser4.marker.size = 8
ser4.marker.graphicalProperties.solidFill = 'FF0000'
ser4.marker.graphicalProperties.line.solidFill = 'FF0000'
ser4.graphicalProperties.line.solidFill = 'FF0000'
ser4.graphicalProperties.line.width = pixels_to_EMU(1.0)

#ser5
ser5=c2.series[1]

ser5.marker.symbol = 'diamond'
ser5.marker.size = 8
ser5.marker.graphicalProperties.solidFill = '008000'
ser5.marker.graphicalProperties.line.solidFill = '008000'
ser5.graphicalProperties.line.solidFill = '008000'
ser5.graphicalProperties.line.width = pixels_to_EMU(1.0)

# #ser6
ser6=c2.series[2]

ser6.marker.symbol = 'square'
ser6.marker.size = 8
ser6.marker.graphicalProperties.solidFill = '0000FF'
ser6.marker.graphicalProperties.line.solidFill = '0000FF'
ser6.graphicalProperties.line.solidFill = '0000FF'
ser6.graphicalProperties.line.width = pixels_to_EMU(1.0)

c2.x_axis.majorTickMark = "in"
c2.y_axis.majorTickMark = "in"
c2.y_axis.majorGridlines = None
c2.x_axis.majorGridlines = None
c2.graphical_properties = GraphicalProperties(ln=LineProperties(noFill=True))
c2.plot_area.spPr = GraphicalProperties(ln = LineProperties(solidFill = "000000"))
c2.x_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))
c2.y_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))

#font
font = Font(typeface='Times New Roman')
size = 1000 # 10 point size
char_prop = CharacterProperties(latin=font, sz=size, b=True, solidFill="000000") #
para_prop = ParagraphProperties(defRPr=char_prop)
c2.x_axis.title.tx.rich.p[0].pPr = para_prop
c2.y_axis.title.tx.rich.p[0].pPr = para_prop

char_prop = CharacterProperties(latin=font, sz=size, b=False) # b=Trueでボールドにする
para_prop = ParagraphProperties(defRPr=char_prop)
rich_text = RichText(p=[Paragraph(pPr=para_prop, endParaRPr=char_prop)])
c2.legend.txPr = rich_text
c2.x_axis.txPr = rich_text
c2.y_axis.txPr = rich_text

Chart1,Chart2,`xlsx`ファイル出力設定

In [8]:
ws.add_chart(c1, "B22")
ws.add_chart(c2,"K22")
# 出力結果ファイル名指定.かぶりを防ぐためケースごとに名前を変更すること.
wb.save("./result/BoxCountResult_noses_cough_x500-1000_phi0_5_1e5_re.xlsx")

`glob`テスト用

In [9]:
import glob
from natsort import natsorted
path_list1 = natsorted(glob.glob("**/open_noses/cough/t20/theta_0phi_5/1e5/*.csv"))
path_list1

['x500\\open_noses\\cough\\t20\\theta_0phi_5\\1e5\\BoxCount.csv',
 'x750\\open_noses\\cough\\t20\\theta_0phi_5\\1e5\\BoxCount.csv',
 'x1000\\open_noses\\cough\\t20\\theta_0phi_5\\1e5\\BoxCount.csv']

In [10]:
import glob
from natsort import natsorted
path_list6 = []
for f in natsorted(glob.glob('x500/open_noses/cough/**/theta_*phi_*')):
    path_list6.append(os.path.split(f)[1])
path_list6

['theta_0phi_0', 'theta_0phi_5', 'theta_0phi_-5']